In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tools.standard_name_utils import optimize_parameters
from tools.common_utils import highlight_diff, read_jsonl

# model_path = "../ChatGLM2-6B/ptuning/output/checkpoint-100"
model_path = "../../train-result/2024-06-19/checkpoint-5000"
top_p=0.7 
temperature=0.95

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████| 6/6 [02:12<00:00, 22.02s/it]


In [3]:
response, history = model.chat(tokenizer, "你好", history=[], top_p=top_p, temperature=temperature)
print(response)

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


你好"}


### 评估一下训练数据的情况

In [ ]:
json_file_path = 'data/train.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

In [ ]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    input = row["input"]
    output = row["output"]
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    total_eval_count += 1
    if response == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, response))
        print(response)
        print()
    
    if total_eval_count == 100:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

In [4]:
json_file_path = 'data/dev.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,input,output
0,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
1,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
2,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
3,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
4,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."


In [5]:
data_stock = pd.read_excel('data/标准名.xlsx',sheet_name='股票标准名')
data_fund = pd.read_excel('data/标准名.xlsx',sheet_name='基金标准名')

fund_standard_name = data_fund['标准基金名称'].to_list()
stock_standard_name = data_stock['标准股票名称'].to_list()

In [6]:
stock_standard_name[:5]

['万马股份', '汉王科技', '珠江啤酒', '联络互动', '奇正藏药']

In [7]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    input = row["input"]
    output = row["output"]
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    
    total_eval_count += 1
    if optimized_resp == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, optimized_resp))
        print(optimized_resp)
        print()
    
    if total_eval_count == 20:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
15
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我想知道我一个月前购买1000元的东兴连众C基金，扣除了销售服务费和认购费用后，我现在的总收益是多少 
 query中提到的产品标准名可能是：华宝中证1000指数分级证券投资基金之稳健收益类份额、华宝中证1000指数分级证券投资基金之积极收益类份额、博道中证1000指数增强型证券投资基金C类、博道中证1000指数增强型证券投资基金A类、银华深证100指数分级证券投资基金之稳健收益类份额、银华深证100指数分级证券投资基金之积极收益类份额、华宝中证1000指数证券投资基金C类、华宝中证1000指数证券投资基金A类、东兴连众一年持有期混合型证券投资基金C类、东兴连众一年持有期混合型证券投资基金A类、长盛同辉深证100等权重指数分级证券投资基金之稳健收益类基金份额、长盛同辉深证100等权重指数分级证券投资基金之积极收益类基金份额、富兰克林国海中证100指数增强型分级证券投资基金之稳健收益类份额、富兰克林国海中证100指数增强型分级证券投资基金之积极收益类份额、鹏华中证1000指数增强型证券投资基金C类、鹏华中证1000指数增强型证券投资基金A类、长信中证1000指数增强型证券投资基金C类、长信中证1000指数增强型证券投资基金A类、财通中证1000指数增强型证券投资基金C类、财通中证1000指数增强型证券投资基金A类、浙商中证1000指数增强型证券投资基金C类、浙商中证1000指数增强型证券投资基金A类、明亚中证1000指数增强型证券投资基金C类、明亚中证1000指数增强型证券投资基金A类、招商中证1000指数增强型证券投资基金C类、招商中证1000指数增强型证券投资基金A类、富国中证1000优选股票型证券投资基金C类、富国中证1000优选股票型证券投资基金A类、太平中证1000指数增强型证券投资基金C类、太平中证1000指数增强型证券投资基金A类、天弘中证1000指数增强型证券投资基金C类、天弘中证1000指数增强型证券投资基金A类、国金中证1000指数增强型证券投资基金C类、国金中证1000指数增强型证券投资基金A类、博时中证1000指数增强型证券投资基金C类、博时中证10

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
19
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我想知道如果我3个月前买入100股海联金汇，然后今天卖出，我需要支付的印花税是多少。 
 query中提到的产品标准名可能是：前海开源股息率100强等权重股票型证券投资基金、前海开源强势共识100强等权重股票型证券投资基金、海联金汇、博道沪深300指数增强型证券投资基金C类、博道沪深300指数增强型证券投资基金A类、富国中证1000优选股票型证券投资基金C类、富国中证1000优选股票型证券投资基金A类、博道中证1000指数增强型证券投资基金C类、博道中证1000指数增强型证券投资基金A类、富国沪深300基本面精选股票型证券投资基金C类、富国沪深300基本面精选股票型证券投资基金A类、招商科技动力3个月滚动持有股票型证券投资基金C类、招商科技动力3个月滚动持有股票型证券投资基金A类、国泰君安中证1000优选股票型发起式证券投资基金C类、国泰君安中证1000优选股票型发起式证券投资基金A类、浙商汇金卓越优选3个月持有期股票型基金中基金(FOF)、同泰积极配置3个月持有期股票型基金中基金(FOF)C类、同泰积极配置3个月持有期股票型基金中基金(FOF)A类、大成中证360互联网+大数据100指数型证券投资基金C类、大成中证360互联网+大数据100指数型证券投资基金A类、长盛中证100指数证券投资基金、融通深证100指数证券投资基金、大成沪深300指数证券投资基金、华富中证100指数证券投资基金、银河定投宝中证腾讯济安价值100A股指数型发起式证券投资基金、银华中国梦30股票型证券投资基金、新疆前海联合海盈货币市场基金B类、新疆前海联合海盈货币市场基金A类、新疆前海联合汇盈货币市场基金B类、新疆前海联合汇盈货币市场基金A类、人保沪深300指数型证券投资基金、南方沪深300交易型开放式指数证券投资基金联接基金A类(后端)、长城消费30股票型证券投资基金C类、长城消费30股票型证券投资基金A类、诺德深证300指数分级证券投资基金、诺安中证100指数证券投资基金C类、诺安中证100指数证券投资基金A类、融通深证100指数证券投资基金C类、招商深证100指数证券投资基金C类、招商深证1

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene

...

-----data index-----
147
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：如果我三年前用5000块买了农银汇理医疗精选C基金，按照销售服务费率算，我现在卖出需要扣除多少费用 
 query中提到的产品标准名可能是：农银汇理医疗精选股票型证券投资基金C类、农银汇理医疗精选股票型证券投资基金A类、农银汇理策略精选混合型证券投资基金、农银汇理创新医疗混合型证券投资基金、农银汇理绿色能源精选混合型证券投资基金、农银汇理医疗保健主题股票型证券投资基金、农银汇理研究精选灵活配置混合型证券投资基金、农银汇理区间精选灵活配置混合型证券投资基金、农银汇理睿选灵活配置混合型证券投资基金、农银汇理景气优选混合型证券投资基金C类、农银汇理景气优选混合型证券投资基金A类、农银汇理均衡优选混合型证券投资基金C类、农银汇理均衡优选混合型证券投资基金A类、农银汇理国企优选混合型证券投资基金C类、农银汇理国企优选混合型证券投资基金A类、农银汇理专精特新混合型证券投资基金C类、农银汇理专精特新混合型证券投资基金A类、农银汇理金禄债券型证券投资基金、农银汇理金硕债券型证券投资基金、农银汇理金盛债券型证券投资基金、农银汇理金益债券型证券投资基金、农银汇理金盈债券型证券投资基金、农银汇理金玉债券型证券投资基金、农银汇理金恒债券型证券投资基金、农银汇理永安混合型证券投资基金、农银汇理天天利货币市场基金B类、农银汇理天天利货币市场基金A类、富国中证500基本面精选股票型发起式证券投资基金C类、富国中证500基本面精选股票型发起式证券投资基金A类、农银汇理货币市场证券投资基金B类、农银汇理货币市场证券投资基金A类、农银汇理红利日结货币市场基金B类、农银汇理红利日结货币市场基金A类、农银汇理中小盘混合型证券投资基金、农银汇理金汇债券型证券投资基金C类、农银汇理金汇债券型证券投资基金A类、农银汇理行业领先混合型证券投资基金、农银汇理策略趋势混合型证券投资基金、农银汇理策略价值混合型证券投资基金、农银汇理新兴消费股票型证券投资基金、农银汇理平衡双利混合型证券投资基金、农银汇理大盘蓝筹混合型证券投资基金、农银汇理均衡收益混合型证券投资基金、农银汇理创新成长混合型证券投资基金、农银汇理信用添利债券型

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
161
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：你们知道哪些基金的一年夏普率是负七十八点四四，并且管理费用比率只有零点一三个百分点的吗？ 
 query中提到的产品标准名可能是：中金优势领航一年持有混合型集合资产管理计划C类、中金丰裕稳健一年持有混合型集合资产管理计划C类、中金丰裕稳健一年持有混合型集合资产管理计划A类、国联金如意双利一年持有期债券型集合资产管理计划C类、国联金如意双利一年持有期债券型集合资产管理计划A类、兴证资管金麒麟领先优势一年持有期混合型集合资产管理计划A类、兴证资管金麒麟恒睿致远一年持有期混合型集合资产管理计划C类、兴证资管金麒麟恒睿致远一年持有期混合型集合资产管理计划A类、兴证资管金麒麟均衡优选一年持有期混合型集合资产管理计划C类、兴证资管金麒麟均衡优选一年持有期混合型集合资产管理计划A类、兴证资管金麒麟兴享优选一年持有期混合型集合资产管理计划C类、兴证资管金麒麟兴享优选一年持有期混合型集合资产管理计划A类、长城淘金一年期理财债券型证券投资基金、博道睿见一年持有期混合型证券投资基金、博道盛兴一年持有期混合型证券投资基金、博道嘉兴一年持有期混合型证券投资基金、中金恒远一年持有期混合型证券投资基金、创金合信群力一年定期开放混合型管理人中管理人(MOM)证券投资基金C类、创金合信群力一年定期开放混合型管理人中管理人(MOM)证券投资基金A类、金鹰景气驱动一年持有期混合型证券投资基金C类、金鹰景气驱动一年持有期混合型证券投资基金A类、金鹰时代领航一年持有期混合型证券投资基金C类、金鹰时代领航一年持有期混合型证券投资基金A类、金鹰年年邮益一年持有期混合型证券投资基金C类、金鹰年年邮益一年持有期混合型证券投资基金A类、金鹰年年邮享一年持有期债券型证券投资基金C类、金鹰年年邮享一年持有期债券型证券投资基金A类、金鹰产业智选一年持有期混合型证券投资基金C类、金鹰产业智选一年持有期混合型证券投资基金A类、海通智选一年持有期股票型集合资产管理计划C类、海通智选一年持有期股票型集合资产管理计划A类、浙商汇金平稳增长一年持有期混合型证券投资基金、国金核心资产一年持有期混合型证券投资基金C类、国金核心资产一年持有期混

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
194
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我上周在地素时尚投入了50000元，假如我已经把股票卖了，减去成本我到底赚还是亏了多少钱 
 query中提到的产品标准名可能是：地素时尚、富国中证1000优选股票型证券投资基金C类、富国中证1000优选股票型证券投资基金A类、富国上证50基本面精选股票型发起式证券投资基金C类、富国上证50基本面精选股票型发起式证券投资基金A类、南方中证500量化增强股票型发起式证券投资基金C类、南方中证500量化增强股票型发起式证券投资基金A类、富国中证500基本面精选股票型发起式证券投资基金C类、富国中证500基本面精选股票型发起式证券投资基金A类、国泰君安中证1000优选股票型发起式证券投资基金C类、国泰君安中证1000优选股票型发起式证券投资基金A类、博时工业4.0主题股票型证券投资基金、上投摩根优选多因子股票型证券投资基金、博时中证500指数增强型证券投资基金C类、博时中证500指数增强型证券投资基金A类、上银中证500指数增强型证券投资基金C类、上银中证500指数增强型证券投资基金A类、博时标普500交易型开放式指数证券投资基金、博时中证500交易型开放式指数证券投资基金、博时中证1000指数增强型证券投资基金C类、博时中证1000指数增强型证券投资基金A类、博时国证2000交易型开放式指数证券投资基金、博时北证50成份指数型发起式证券投资基金C类、博时北证50成份指数型发起式证券投资基金A类、博时中证2000交易型开放式指数证券投资基金、前海开源港股通股息率50强股票型证券投资基金、中信建投中证500指数增强型证券投资基金C类、中信建投中证500指数增强型证券投资基金A类、上证周期行业50交易型开放式指数证券投资基金、汇添富上证50基本面增强指数型证券投资基金C类、汇添富上证50基本面增强指数型证券投资基金A类、富国沪深300基本面精选股票型证券投资基金C类、富国沪深300基本面精选股票型证券投资基金A类、前海开源股息率100强等权重股票型证券投资基金、中信建投中证1000指数增强型证券投资基金C类、中信建投中证1000指数增强型证券投资基金A类、汇添富中证500基本面增

In [ ]:
test_input = df.loc[35]["input"]
test_input

In [ ]:
test_input = df.loc[19]["input"]
test_input += " 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价"
response, history = model.chat(tokenizer=tokenizer, query=test_input, history=[], top_p=top_p, temperature=temperature)
optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)

output = df.loc[19]["output"]
print(highlight_diff(output, optimized_resp))
print(optimized_resp)

### 输出新的提交文件

In [8]:
test_json_file_path = 'data/test_a.json'
test_data_list = read_jsonl(test_json_file_path)

test_df = pd.DataFrame(test_data_list)
test_df.head()

,input,output
0,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock
1,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock
2,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock
3,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock
4,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,mock


In [9]:
total_eval_count = 0

with open('data/submit.txt','w', encoding="utf-8") as n:
    for index, row in test_df.iterrows():    
        input = row["input"]
        response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
        
        optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
        
        total_eval_count += 1
        n.write(optimized_resp+'\n')
        if total_eval_count % 10 == 0:
            print("现在是第" + f"{total_eval_count}" + "条数据")

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene

现在是第10条数据


Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene

现在是第20条数据


Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene

...

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene

现在是第1260条数据


Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene

现在是第1270条数据


Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
